# Basic RNN cell

In [1]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Parameters
learning_rate = 0.001
training_iters = 50000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 1 # MNIST data input (img shape: 28*28)
n_steps = 784 # timesteps
n_hidden = 100 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = tf.Variable(tf.random_normal([n_hidden, n_classes]))
biases = tf.Variable(tf.random_normal([n_classes]))

# Permuting batch_size and n_steps
tmp_x = tf.transpose(x, [1, 0, 2])
# Reshaping to (n_steps*batch_size, n_input)
tmp_x = tf.reshape(tmp_x, [-1, n_input])
# Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
tmp_x = tf.split(0, n_steps, tmp_x)

# Define a lstm cell with tensorflow
lstm_cell = rnn_cell.BasicRNNCell(n_hidden)

# Get lstm cell output
outputs, states = rnn.rnn(lstm_cell, tmp_x, dtype=tf.float32)
pred = tf.matmul(outputs[-1], weights) + biases

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_data = mnist.test.images.reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
    
tf.reset_default_graph()

Iter 10, Training Accuracy= 0.04688
Iter 20, Training Accuracy= 0.10938
Iter 30, Training Accuracy= 0.11719
Iter 40, Training Accuracy= 0.08594
Iter 50, Training Accuracy= 0.10938
Iter 60, Training Accuracy= 0.07812
Iter 70, Training Accuracy= 0.10938
Iter 80, Training Accuracy= 0.09375
Iter 90, Training Accuracy= 0.10156
Iter 100, Training Accuracy= 0.14062
Iter 110, Training Accuracy= 0.09375
Iter 120, Training Accuracy= 0.07812
Iter 130, Training Accuracy= 0.14844
Iter 140, Training Accuracy= 0.05469
Iter 150, Training Accuracy= 0.09375
Iter 160, Training Accuracy= 0.14844
Iter 170, Training Accuracy= 0.10938
Iter 180, Training Accuracy= 0.14844
Iter 190, Training Accuracy= 0.08594
Iter 200, Training Accuracy= 0.08594
Iter 210, Training Accuracy= 0.10156
Iter 220, Training Accuracy= 0.07031
Iter 230, Training Accuracy= 0.10156
Iter 240, Training Accuracy= 0.10938
Iter 250, Training Accuracy= 0.07031
Iter 260, Training Accuracy= 0.15625
Iter 270, Training Accuracy= 0.14844
Iter 280, 